# Итоговое задание


Это ваше финальное задание, посвященное пространственному анализу одного из проектов нового жилого строительства.

## Где взять данные?

Для работы нужно выбрать любую из новостроек России, данные можно найти:

- [В каталоге новостроек ДОМА.РФ](https://наш.дом.рф/сервисы/каталог-новостроек/список-объектов/карта?bounds=55.67696035623858%2C37.27774042429143%2C55.81697009367747%2C37.7968444281977)
- Участок не должен совпадать с примером ниже
- можно выбрать любой другой интересующий вас проект

--

## Требования к содержанию работы

1. **Краткое описание исследуемого проекта**  
   Укажите основные характеристики проекта (в каком городе находится, в каком районе, год ввода в эксплуатацию, застройщик)

2. **Расчет показателей** с комментариями (что именно считаете, как)
   В работе должно быть рассчитано **не менее пяти показателей**, из которых **два** должны быть основаны на данных, созданных вами в результате пространственного анализа.

   ### Примеры показателей:

   - **Простой расчет**: Анализ текущей структуры землепользования по долям (расчет на основе исходных данных без применения дополнительных инструментов пространственного анализа).
   - **Показатель на основе пространственного анализа**: Оценка доступности объектов социальной инфраструктуры (с использованием методов пространственного анализа, таких как изохроны или расчет расстояний).

3. **Краткое описание результатов и выводы**  
   Подведите итоги анализа, сделайте выводы о пространственном расположении проекта и его особенностях

--

## Какие показатели можно включить в анализ?

Вот некоторые идеи для анализа (вы можете выбрать свои релевантные показатели):

- Анализ текущей структуры землепользования
- Расчет коэффициента застройки территории (доли застроенной территории)
- Оценка удаленности от центра города или других важных объектов
- Оценка существующей плотности населения
- Доступность объектов социальной инфраструктуры
- Доступность общественного транспорта
- Доступность рекреационных зон
- Плотность сервисов и коммерческих услуг
- **И любые другие показатели**, которые кажутся вам значимыми для задачи.

--

## Формат сдачи

1. Jupyter Notebook как отчёт - выполненные шаги, комментарии, выводы
2. Опубликованная интерактивная карта со всеми слоями - результатом работы, на карту обязательно добавить Layer Control (самый простой способ посмотрели на паре -- опубликовать на `gh-pages`, но можно использовать другие известные способы)

--

## Консультация:

На последней встрече **03.12** (или 10.12) (?) мы проведем консультацию, где можно будет задать любые вопросы по заданию и по работе с пространственными данными в целом.

## Дедлайн:

**Официальный срок сдачи:** до **14.12** в **23:59**.


---

## Начало работы


Импортируем библиотеки


In [1]:
import json
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium

from IPython.display import Markdown

ox.settings.log_console = False
ox.settings.use_cache = True

[В каталоге новостроек ДОМА.РФ](https://наш.дом.рф/сервисы/каталог-новостроек/список-объектов/карта?bounds=55.67696035623858%2C37.27774042429143%2C55.81697009367747%2C37.7968444281977) находим интересующий нас объект и узнаем его ID

![ID](assets/id.png)


По следующему url могу получить информацию об объекте


In [2]:
base_url = "https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api"

# ЖК "Среда на Кутузовском"
test_id = "60622"

url_get_point_info = f"{base_url}/object/{test_id}"


In [3]:
url_get_point_info

'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/60622'

На основе полученного json и создаем GeoDataFrame (для примера сделала запрос в браузере и скопировала в файл assets/object.json)


In [4]:
with open("object.json", "r", encoding="utf-8") as f:
    data = json.load(f)

Сохраняем полезную информацию


In [5]:
data = data['data']

jk_features  = {
    "name": data["nameObj"],
    "address": data["address"],
    "class": data["objLkClassDesc"],
    "floors_min": data["floorMin"],
    "floors_max": data["floorMax"],
    "flats_total": data["objFlatCnt"],
    "living_square": float(data["objSquareLiving"]),
    "parking_cnt": data["objElemParkingCnt"],
    "price_min": data["freeFlatsInfo"]["priceMin"],
    "price_avg": data["objPriceAvg"],
    "developer": data["developer"]["developerGroupName"],
    "latitude": float(data["objLkLatitude"]),
    "longitude": float(data["objLkLongitude"]),
}

df = pd.DataFrame([jk_features])

gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_xy(df['longitude'], df['latitude']), crs='EPSG:4326')

gdf.explore(tiles='cartodbpositron')


In [6]:
gdf

,name,address,class,floors_min,floors_max,flats_total,living_square,parking_cnt,price_min,price_avg,developer,latitude,longitude,geometry
0,"ЖК ""Среда на Кутузовском""","ГОРОД МОСКВА, Район Кунцево",Бизнес,14,46,785,41893.8,347,19131904,502954,ПИК,55.7285,37.4331,POINT (37.4331 55.7285)


# 1. Описание исследуемого проекта

Объект исследования — жилой комплекс **«Среда на Кутузовском»**, расположенный в городе **Москва**, район **Кунцево**

**Основные характеристики проекта:**
- Адрес: город Москва, улица Баррикадная, дом 19, строение 1
- Застройщик: АО СЗ МОНЕТЧИК (ГК ПИК)
- Ввод в эксплуатацию: 4Q2027
- Класс жилья: Бизнес


# 2. Расчет показателей


In [7]:
apart_sq_avg = round((gdf['living_square'] / gdf['flats_total'])[0], 2)
Markdown('**Средняя площадь квартиры:** ' + str(apart_sq_avg) + ' м²')

**Средняя площадь квартиры:** 53.37 м²

In [8]:
ratio = round((gdf['flats_total'] / gdf['parking_cnt'])[0], 1)
Markdown('**Сколько квартир приходится на одно парковочное место:** ' + str(ratio))

**Сколько квартир приходится на одно парковочное место:** 2.3

In [9]:
utm_crs = gdf.estimate_utm_crs()

In [10]:
jk_point_m = gdf.to_crs(utm_crs)

# Буфер 1000 м
jk_buffer_m = jk_point_m.copy()
jk_buffer_m["geometry"] = jk_buffer_m.geometry.buffer(1000)

jk_buffer_wgs = jk_buffer_m.to_crs(epsg=4326)


In [11]:
tags = {
    "amenity": ["cafe", "school", "kindergarten", "hospital", "clinic"],
    "leisure": ["fitness_centre"],
    "railway": ["subway_entrance"]
}

pois = ox.features_from_polygon(
    jk_buffer_wgs.geometry.iloc[0],
    tags=tags
)

pois = pois.to_crs(utm_crs)


In [12]:
def classify_poi(row):
    if row.get("amenity") == "cafe":
        return "cafe"
    if row.get("leisure") == "fitness_centre":
        return "gym"
    if row.get("amenity") == "school":
        return "school"
    if row.get("amenity") == "kindergarten":
        return "kindergarten"
    if row.get("amenity") in ["hospital"]:
        return "hospital"
    if row.get("amenity") in ["clinic"]:
        return "clinic"
    if row.get("railway") == "subway_entrance":
        return "metro"
    return None

pois["poi_type"] = pois.apply(classify_poi, axis=1)
pois = pois.dropna(subset=["poi_type"])


In [13]:
jk_geom = jk_point_m.geometry.iloc[0]

rows = []

for poi_type, group in pois.groupby("poi_type"):
    distances = group.geometry.distance(jk_geom)
    rows.append({
        "poi_type": poi_type,
        "count": len(group),
        "min_distance_m": round(distances.min(), 1)
    })

infra_df = pd.DataFrame(rows).sort_values("poi_type")
infra_df


,poi_type,count,min_distance_m
0,cafe,1,926.9
1,clinic,5,790.0
2,gym,1,297.7
3,hospital,1,578.4
4,kindergarten,9,344.8
5,metro,7,807.6
6,school,11,133.0


In [14]:
from folium.plugins import MousePosition, Fullscreen

# --- подготовка POI для карты ---
pois_map = pois.to_crs(epsg=4326).copy()

# центроиды (для отображения полигонов)
pois_map["geometry"] = pois_map.geometry.centroid

# расстояние до ЖК (в метрах)
pois_map["distance_m"] = pois.geometry.distance(jk_geom).round(0).astype(int)

# названия типов объектов
poi_type_ru = {
    "cafe": "Кафе",
    "gym": "Спортзал",
    "school": "Школа",
    "kindergarten": "Детский сад",
    "hospital": "Больница",
    "clinic": "Поликлиника",
    "metro": "Метро"
}

# стили объектов
poi_style = {
    "cafe": {"color": "orange", "icon": "coffee"},
    "gym": {"color": "gray", "icon": "dumbbell"},
    "school": {"color": "blue", "icon": "graduation-cap"},
    "kindergarten": {"color": "green", "icon": "child"},
    "hospital": {"color": "purple", "icon": "plus"},
    "clinic": {"color": "pink", "icon": "medkit"},
    "metro": {"color": "darkblue", "icon": "subway"},
}

# название объекта из OSM
def get_osm_name(row):
    if pd.notna(row.get("name")):
        return row.get("name")
    return poi_type_ru.get(row.poi_type, "Объект инфраструктуры")

pois_map["osm_name"] = pois_map.apply(get_osm_name, axis=1)

# --- карта ---
m = folium.Map(
    location=[jk_features["latitude"], jk_features["longitude"]],
    zoom_start=15,
    tiles="cartodbpositron"
)

# --- ЖК ---
folium.Marker(
    [jk_features["latitude"], jk_features["longitude"]],
    popup=folium.Popup(
        f"<b>{jk_features['name']}</b><br>{jk_features['address']}",
        max_width=300
    ),
    icon=folium.Icon(color="red", icon="home", prefix="fa")
).add_to(m)

# --- буфер 1000 м ---
folium.GeoJson(
    jk_buffer_wgs,
    name="Буфер 1000 м",
    style_function=lambda x: {
        "fillOpacity": 0.1,
        "color": "#3186cc",
        "weight": 2
    }
).add_to(m)

# --- POI ---
for poi_type, group in pois_map.groupby("poi_type"):
    style = poi_style.get(poi_type, {"color": "gray", "icon": "info-sign"})
    fg = folium.FeatureGroup(name=poi_type_ru.get(poi_type, poi_type))

    for _, row in group.iterrows():
        popup_html = f"""
        <b>{row.osm_name}</b><br>
        Тип: {poi_type_ru.get(poi_type, poi_type)}<br>
        Расстояние до ЖК: {row.distance_m} м
        """

        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(
                color=style["color"],
                icon=style["icon"],
                prefix="fa"
            )
        ).add_to(fg)

    fg.add_to(m)

# --- контролы ---
folium.LayerControl(collapsed=False).add_to(m)

MousePosition(
    position="bottomright",
    separator=" | ",
    prefix="Координаты:",
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}"
).add_to(m)

Fullscreen(
    position="topright",
    title="На весь экран",
    title_cancel="Выйти из полноэкранного режима",
    force_separate_button=True
).add_to(m)

m


C:\Users\chizh\AppData\Local\Temp\ipykernel_30460\1744711316.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois_map["geometry"] = pois_map.geometry.centroid


In [15]:
m.save("index.html")

**Интерактивная карта доступна по ссылке:**

https://mikhailchizhmar.github.io/InteractiveMap/

# 3. Выводы

В рамках работы был проведён анализ пространственного положения жилого комплекса **«Среда на Кутузовском»**, расположенного в районе Кунцево города Москвы. Для анализа использовались как атрибутивные характеристики проекта, так и данные пространственного анализа в радиусе **1000 метров** от объекта.

По результатам расчётов средняя площадь квартиры в проекте составляет около **53 м²**, что соответствует заявленному бизнес-классу жилья. Соотношение количества квартир к числу парковочных мест составляет примерно **2,3 квартиры на одно машиноместо**, что указывает на слабо-умеренную обеспеченность парковкой и потенциальную нагрузку на уличную парковку в будущем.

Пространственный анализ доступности инфраструктуры показал, что в пределах пешеходной доступности присутствует **разнообразный набор объектов социальной и сервисной инфраструктуры**. В радиусе 1000 метров расположены кафе, спортивные объекты, школы, детские сады, медицинские учреждения и станции метро. Минимальные расстояния до ключевых объектов инфраструктуры составляют **сотни метров**, что говорит о высокой степени интеграции жилого комплекса в существующую городскую среду. 

Лично для меня большим плюсом является расстояние до спортивного зала - он находится в 300 метрах от ЖК, но в то же время меня огорчает расположение ближайшего кафе - аж в 900 метрах от ЖК на другой стороне шоссе.

Особенно важно отметить наличие **станций метро в пешей доступности**, что положительно влияет на транспортную доступность района и снижает зависимость жителей от личного транспорта. Также в окружении объекта присутствует достаточное количество образовательных и медицинских учреждений, что делает проект привлекательным для семей с детьми.

В целом, результаты анализа показывают, что жилой комплекс расположен в **сформированной городской среде** с хорошей транспортной и социальной доступностью. Пространственное положение проекта можно оценить как благоприятное, а уровень инфраструктурной обеспеченности — как высокий для рассматриваемого класса жилья.